In [ ]:
adls_name = "datainsightsadhoc" # Change this to your ADLS Accountg
tenant_id = "microsoft.onmicrosoft.com" # Change this to your AD tenant

In [ ]:
import azure.datalake.store

In [ ]:
def get_token_prompt(tenant_id):
    import adal
    import time
    authority = 'https://login.microsoftonline.com/' # LEAVE THIS ALONE
    resource = 'https://management.core.windows.net/' # LEAVE THIS ALONE
    client_id = '1950a258-227b-4e31-a9cf-717495945fc2' # THIS IS THE AZURE POWERSHELL CLIENTID
    
    context = adal.AuthenticationContext(authority + tenant_id)
    code = context.acquire_user_code(resource, client_id)
    print(code['message'])
    token = context.acquire_token_with_device_code(resource, code, client_id)
    token.update({'access': token['accessToken'], 'resource': resource,
                'refresh': token.get('refreshToken', False),
                'time': time.time(), 'tenant': tenant_id, 'client': client_id})
    return token

In [ ]:
load_token = True
save_token = False
token_filename = 'd:\\adlstoken.pickle'

import os
import pickle


if (load_token and os.path.exists(token_filename)) :
    token = pickle.load( open( token_filename, 'rb') )
else :        
    token = get_token_prompt(tenant_id)
    
if (save_token) :
    pickle.dump( token, open( token_filename, 'wb') )

In [ ]:
print(token)

In [ ]:
print(token.keys())

In [ ]:
fsclient = azure.datalake.store.core.AzureDLFileSystem(token, store_name="datainsightsadhoc")


In [ ]:
dir(fsclient)

In [ ]:

fsclient.ls()

In [ ]:
fsclient.mkdir('/python_sdk_demo')

In [ ]:
fsclient.ls('/python_sdk_demo')

In [ ]:
import os

if (not (os.path.exists('d:\\test.txt'))) :
    f = open( 'd:\\test.txt', 'w')
    f.write("Hello World")
    f.close()

In [ ]:

if (fsclient.exists( '/python_sdk_demo/test.txt' )) :
    fsclient.rm( '/python_sdk_demo/test.txt' )
    
azure.datalake.store.multithread.ADLUploader(fsclient, lpath='d:\\test.txt', rpath='/python_sdk_demo/test.txt')

In [ ]:
fsclient.ls('/python_sdk_demo')

In [ ]:
fsclient.rm('/python_sdk_demo', recursive=True)

In [ ]:
fsclient.ls()